# Making a Connection with MongoClient

The first thing we need to do is import `pymongo`. The import should run without any errors to signify that we've done our installation well.

In [26]:
import pymongo
from bson.objectid import ObjectId

Establishing a connection in MongoDB requires us to create a `MongoClient` to the running MongoDB instance. We can specify the MongoDB's host and port as shown below:

In [2]:
from pymongo import MongoClient
client = MongoClient('mongodb://192.168.103.46:27017/')

# Creating a Database
To create a database in MongoDB, we use the `MongoClient` instance and specify a database name. MongoDB will create a database if it doesn't exist and connect to it.

In [3]:
db = client['bigdatadb']

note: You need to replace the `bigdatadb` with your own database name.

It is important to note that databases and collections are created lazily in MongoDB. This means that the collections and databases are created when the first document is inserted into them.

# Data in MongoDB

Data in MongoDB is represented and stored using JSON-Style documents. In PyMongo we use dictionaries to represent documents. Let's show an example of a PyMongo document below:

In [4]:
lecture = {"lecturer": "Muhammad Farhan Sjaugi", "lecture": "Introduction to MongoDB and Python", "tags": ["mongodb", "python", "pymongo"]}

## Inserting a Document

To insert a document into a collection, we use the `insert_one()` method. As we saw earlier, a collection is similar to a table in RDBMS while a document is similar to a row.

In [5]:
courses = db.courses
result = courses.insert_one(lecture)

When the document is inserted, a special key `_id` is generated and its unique to this document. We can print the document ID as shown below:

In [6]:
print("First lecture key is: {}".format(result.inserted_id))

First lecture key is: 5fce3b9d5f48af72bc8bb6dc


The courses collection is created after inserting the first document. We can confirm this using the `list_collection_names` method.

In [7]:
db.list_collection_names()

['courses']

We can insert multiple documents to a collection using the `insert_many()` method as shown below.

In [8]:
lecture1 = {"lecturer": "Muhammad Farhan Sjaugi","lecture": "Hadoop and Spark","tags":["hadoop","spark"]}
lecture2 = {"lecturer": "Muhammad Farhan Sjaugi","lecture": "Python and NoSQL","tags":["python","nosql"]}
new_lectures = courses.insert_many([lecture1, lecture2])
print("The new lecture IDs are {}".format(new_lectures.inserted_ids))

The new lecture IDs are [ObjectId('5fce3cbe5f48af72bc8bb6dd'), ObjectId('5fce3cbe5f48af72bc8bb6de')]


## Retrieving a Single Document with **find_one()**
`find_one()` returns a single document matching the query or none if it doesn't exist. This method returns the first match that it comes across. When we call the method below, we get the first lecture we inserted into our collection.

In [9]:
print(courses.find_one())

{'_id': ObjectId('5fce3b9d5f48af72bc8bb6dc'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python', 'tags': ['mongodb', 'python', 'pymongo']}


## Finding all Documents in a Collection
MongoDB also allows us to retrieve all documents in a collection using the `find` method.

In [11]:
for lecture in courses.find():
  print(lecture)

{'_id': ObjectId('5fce3b9d5f48af72bc8bb6dc'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5fce3cbe5f48af72bc8bb6dd'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Hadoop and Spark', 'tags': ['hadoop', 'spark']}
{'_id': ObjectId('5fce3cbe5f48af72bc8bb6de'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Python and NoSQL', 'tags': ['python', 'nosql']}


## Return Some Fields Only
Sometimes we might not want to return all the fields from our documents. Let's show we'd fetch specific fields. In our case we use 0 to specify that the `_id` should not be fetched and 1 to specify that `lecturer` and `lecture` should be fetched. MongoDB doesn't allow us to specify zero twice. For example, specify `tags` to 0 below will generate an error. We are not allowed to specify both 0 and 1 values in the same object (unless one of the fields is the `_id` field). When we specify a field with the value 0, all other fields get the value 1.

In [12]:
for lecture in courses.find({},{ "_id": 0, "lecturer": 1, "lecture": 1}):
  print(lecture)

{'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python'}
{'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Hadoop and Spark'}
{'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Python and NoSQL'}


## Sorting the Results
We can use the `sort()` method to sort the results in ascending or descending order. The default order is ascending. We use 1 to signify ascending and -1 to signify descending.

In [14]:
doc = courses.find().sort("lecture", -1)

for x in doc:
  print(x)

{'_id': ObjectId('5fce3cbe5f48af72bc8bb6de'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Python and NoSQL', 'tags': ['python', 'nosql']}
{'_id': ObjectId('5fce3b9d5f48af72bc8bb6dc'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5fce3cbe5f48af72bc8bb6dd'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Hadoop and Spark', 'tags': ['hadoop', 'spark']}


## Updating a Document
We update a document using the `update_one()` method. The first parameter taken by this function is a query object defining the document to be updated. If the method finds more than one document, it will only update the first one. Let's update the name of the lecturer in the lecture with topics 'Python and NoSQL'

In [25]:
query = { "lecture": "Python and NoSQL" }
new_lecturer = { "$set": { "lecturer": "Mohammad Asif Khan" } }

courses.update_one(query, new_lecturer)

for lecture in courses.find():
  print(lecture)

{'_id': ObjectId('5fce3b9d5f48af72bc8bb6dc'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5fce3cbe5f48af72bc8bb6dd'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Hadoop and Spark', 'tags': ['hadoop', 'spark']}
{'_id': ObjectId('5fce3cbe5f48af72bc8bb6de'), 'lecturer': 'Mohammad Asif Khan', 'lecture': 'Python and NoSQL', 'tags': ['python', 'nosql']}


## Limiting the Result
MongoDB enables us to limit the result of our query using the `limit` method. In our query below we'll limit the result to one record.

In [22]:
limited_result = courses.find().limit(1)
for x in limited_result:
    print(x)

{'_id': ObjectId('5fce3b9d5f48af72bc8bb6dc'), 'lecturer': 'Muhammad Farhan Sjaugi', 'lecture': 'Introduction to MongoDB and Python', 'tags': ['mongodb', 'python', 'pymongo']}


## MongoDB Delete Document
We use the `delete_one()` method to delete a document in MongoDB. The first parameter for this method is the query object of the document we want to delete. If this method finds more than one document, it deletes only the first one found. Let's delete the article with the id `5fce3b9d5f48af72bc8bb6dc`

In [27]:
db.courses.delete_one({"_id":ObjectId("5fce3b9d5f48af72bc8bb6dc")})

## Deleting Many Documents
In order to delete many documents, we use the `delete_many()` method. Passing an empty query object will delete all the documents.

In [28]:
delete_lectures = courses.delete_many({})
print(delete_lectures.deleted_count, " lectures deleted.")

2  lectures deleted.


## Dropping a Collection
In MongoDB, we can delete a collection using the `drop()` method.

In [29]:
courses.drop()

We can confirm that the collection has been deleted since when we call the `list_collection_names`, we get an empty list.

In [30]:
db.list_collection_names()

[]